## Overall Workflow

1. Initialize regressors 
2. Build the pipelines
3. Set up the parameter grids
4. Set up multiple GridSearchCV for the hyperparameter tuning process of each algorithm. This takes place in the inner loop.
5. Define the inner and outer loops
* Use StratifiedKFold to create outer loop folds
* Training folds created in the outerloop will be used in the inner loop for parameter tuning.
* The inner loop selects the best hyperparameter setting. The best hyperparameter will be evaluated on both the average across inner test folds and the *one* corresponding test fold of the outer loop.

In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

#Set random seed to current date and time to get a pseudorandom state
import random
from datetime import datetime
random.seed(datetime.now()) 

#This is to round the output to 3 decimal places when printing outputs
np.set_printoptions(precision=3)


In [2]:

#Import dataset
data = pd.read_csv("C:/Users/kimng/Desktop/ML - Age, Hipp, CVLT/CVLTHippocampus.csv")
data = data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Columns: 206 entries, ID_code to RecallEfficiency
dtypes: float64(181), int64(22), object(3)
memory usage: 225.4+ KB


In [3]:
data.columns

Index(['ID_code', 'Scan_code', 'Sex', 'Age', 'Age50', 'AgeCategory', 'ICV',
       'Handedness', 'Education', 'EduYears',
       ...
       'CVLT_CorrectRate_B', 'CVLT_CorrectRate_SDFR', 'CVLT_CorrectRate_SDCR',
       'CVLT_CorrectRate_LDFR', 'CVLT_CorrectRate_LDCR', 'GenVerbLearn',
       'ResponseError', 'OrgStrat', 'SerialEffect', 'RecallEfficiency'],
      dtype='object', length=206)

In [4]:
columns = ['CVLT_Imm_Total', 'CVLT_DelR_SD_Free', 'CVLT_DelR_LD_Free',
            'Age','Sex', 'EduYears', 'Smoker', 'High_BP', 'COMT', 'BDNF2', 'ApoE4',
           'L_HH_Total', 'R_HH_Total', 'L_HB_Total', 'R_HB_Total', 'L_HT_Total', 'R_HT_Total',
           'L_DG_Total', 'R_DG_Total',
           'L_CA_Total', 'R_CA_Total',
           'L_Sub_Total', 'R_Sub_Total',
           'L_HH_CA', 'R_HH_CA', 'L_HB_CA', 'R_HB_CA', 'L_HT_CA', 'R_HT_CA', 
           'L_HH_DG', 'R_HH_DG', 'L_HB_DG', 'R_HB_DG', 'L_HT_DG', 'R_HT_DG',
           'L_HH_Sub', 'R_HH_Sub', 'L_HB_Sub', 'R_HB_Sub', 'L_HT_Sub', 'R_HT_Sub']


In [5]:
df = data[columns]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CVLT_Imm_Total     137 non-null    float64
 1   CVLT_DelR_SD_Free  137 non-null    float64
 2   CVLT_DelR_LD_Free  137 non-null    float64
 3   Age                140 non-null    int64  
 4   Sex                140 non-null    int64  
 5   EduYears           140 non-null    int64  
 6   Smoker             140 non-null    int64  
 7   High_BP            140 non-null    int64  
 8   COMT               140 non-null    int64  
 9   BDNF2              140 non-null    int64  
 10  ApoE4              140 non-null    int64  
 11  L_HH_Total         129 non-null    float64
 12  R_HH_Total         129 non-null    float64
 13  L_HB_Total         129 non-null    float64
 14  R_HB_Total         129 non-null    float64
 15  L_HT_Total         129 non-null    float64
 16  R_HT_Total         129 non

In [9]:
df.dropna(inplace=True)

#reset index -- this is to replace old data index with index based on current data.
df = df.reset_index(drop=True)

# get new data dimension
df.shape 

(129, 41)

In [10]:
df.Sex = df.Sex-1

### Create category variable for stratification

In [13]:
#Bin Age into groups
bins = [10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = [1,2,3,4,5,6,7,8]
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels)
df = df.reset_index(drop=True)

#Function for categorise dataframe
def categorise(row):
    if row['AgeGroup'] == 1 and row['Sex'] == 0:
        return 1
    elif row['AgeGroup'] == 1 and row['Sex'] == 1:
        return 2
    elif row['AgeGroup'] == 2 and row['Sex'] == 0:
        return 3
    elif row['AgeGroup'] == 2 and row['Sex'] == 1:
        return 4
    elif row['AgeGroup'] == 3 and row['Sex'] == 0:
        return 5
    elif row['AgeGroup'] == 3 and row['Sex'] == 1:
        return 6
    elif row['AgeGroup'] == 4 and row['Sex'] == 0:
        return 7
    elif row['AgeGroup'] == 4 and row['Sex'] == 1:
        return 8
    elif row['AgeGroup'] == 5 and row['Sex'] == 0:
        return 9
    elif row['AgeGroup'] == 5 and row['Sex'] == 1:
        return 10
    elif row['AgeGroup'] == 6 and row['Sex'] == 0:
        return 11
    elif row['AgeGroup'] == 6 and row['Sex'] == 1:
        return 12
    elif row['AgeGroup'] == 7 and row['Sex'] == 0:
        return 13
    elif row['AgeGroup'] == 7 and row['Sex'] == 1:
        return 14
    elif row['AgeGroup'] == 8 and row['Sex'] == 0:
        return 15
    elif row['AgeGroup'] == 8 and row['Sex'] == 1:
        return 16


#Apply categories to dataframe
df['grp'] = df.apply(lambda row: categorise(row), axis=1)

In [14]:
df.head()

,index,CVLT_Imm_Total,CVLT_DelR_SD_Free,CVLT_DelR_LD_Free,Age,Sex,EduYears,Smoker,High_BP,COMT,...,L_HT_DG,R_HT_DG,L_HH_Sub,R_HH_Sub,L_HB_Sub,R_HB_Sub,L_HT_Sub,R_HT_Sub,AgeGroup,grp
0,0,63.0,15.0,16.0,31,1,16,1,1,1,...,299.478210,285.051172,285.855522,307.093904,208.841138,263.115850,34.753172,38.936464,3,6
1,1,52.0,12.0,12.0,50,0,18,1,1,3,...,216.331552,194.072584,287.146142,331.563567,189.983780,215.776155,18.675363,31.495736,4,7
2,2,60.0,15.0,15.0,30,1,16,1,1,3,...,296.479599,249.553920,327.550589,353.394837,229.576124,298.222287,43.905748,38.330404,2,4
3,3,54.0,12.0,11.0,67,1,13,1,1,1,...,171.081315,161.054810,249.491163,361.781214,238.156421,215.735688,24.102164,30.143226,6,12
4,4,67.0,15.0,14.0,19,1,14,1,1,2,...,252.891192,267.005147,230.117166,296.707669,232.618900,173.662080,30.216082,29.959485,1,2


In [15]:
df.describe()

,index,CVLT_Imm_Total,CVLT_DelR_SD_Free,CVLT_DelR_LD_Free,Age,Sex,EduYears,Smoker,High_BP,COMT,...,R_HB_DG,L_HT_DG,R_HT_DG,L_HH_Sub,R_HH_Sub,L_HB_Sub,R_HB_Sub,L_HT_Sub,R_HT_Sub,grp
count,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,...,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000
mean,64.000000,54.775194,11.666667,12.248062,47.635659,0.542636,15.860465,1.031008,1.116279,2.062016,...,312.167117,215.278658,216.086355,270.114788,286.853054,216.401042,208.797596,26.868606,29.594662,8.054264
std,37.383151,9.126062,2.608080,2.613161,18.883251,0.500121,2.461403,0.174014,0.321809,0.736885,...,56.639130,55.946779,57.863672,54.831992,53.460985,41.213681,35.336911,6.789627,8.055470,3.863511
min,0.000000,35.000000,6.000000,5.000000,18.000000,0.000000,10.000000,1.000000,1.000000,1.000000,...,173.639405,89.231806,89.795907,148.819290,158.846793,110.909966,106.781205,12.233782,15.553328,1.000000
25%,32.000000,49.000000,10.000000,10.000000,30.000000,0.000000,14.000000,1.000000,1.000000,2.000000,...,271.435985,174.712057,181.698029,229.770510,254.843688,186.129439,187.518921,22.200925,24.290453,4.000000
50%,64.000000,55.000000,12.000000,12.000000,48.000000,1.000000,16.000000,1.000000,1.000000,2.000000,...,313.215662,213.735190,209.295573,265.785139,285.320587,215.537167,209.140882,24.817369,28.900795,8.000000
75%,96.000000,61.000000,14.000000,14.000000,64.000000,1.000000,17.000000,1.000000,1.000000,3.000000,...,349.277249,254.920021,254.615139,310.839873,317.390520,243.406118,230.304132,31.469792,33.466886,11.000000
max,128.000000,73.000000,16.000000,16.000000,85.000000,1.000000,23.000000,2.000000,2.000000,3.000000,...,455.998451,352.813403,363.037216,433.877967,494.619458,332.358149,305.395180,46.911832,55.632057,16.000000


### Initialize Regressors

In [16]:
reg1 = Ridge()
reg2 = Lasso()
reg3 = ElasticNet()
reg4 = KNeighborsRegressor()
reg5 = SVR()
reg6 = DecisionTreeRegressor()
reg7 = RandomForestRegressor()

### Build the pipelines
for Ridge, Lasso, ElasticNet, K-Neighbors, & SVM because feature scaling is required for these algorithms.

In [17]:
#pipe1 = Pipeline([
#                 #('scaler', MinMaxScaler()),
#                  ('reg1', reg1)])
#pipe2 = Pipeline([
#                 #('scaler', MinMaxScaler()),
#                  ('reg2', reg2)])
#pipe3 = Pipeline([
#                 #('scaler', MinMaxScaler()),
#                  ('reg3', reg3)])
#pipe4 = Pipeline([
#                 #('scaler', MinMaxScaler()),
#                  ('reg4', reg4)])
#pipe5 = Pipeline([
#                 #('scaler', MinMaxScaler()),
#                  ('reg5', reg5)])
#pipe6 = Pipeline([
                 #('scaler', MinMaxScaler()),
#                  ('reg6', reg6)])
#pipe7 = Pipeline([
#                 #('scaler', MinMaxScaler()),
#                  ('reg7', reg7)])


### Setting up the parameter grids

In [18]:
param_grid1 = [{'alpha': [.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 15, 20]}]

param_grid2 = [{'alpha': [.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 15, 20]}]

param_grid3 = [{'alpha': [.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 15, 20]}]

param_grid4 = [{'n_neighbors': list(range(1, 10))}]

param_grid5 = [{'C': [0.01, 0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, 40, 50], 
                'gamma': ['scale', 'auto'], 
                'kernel': ['rbf', 'linear', 'poly'], 
                'degree': [1,2,3,4,5,6]}]

param_grid6 = [{'criterion': ['absolute_error'],
                'max_depth': [5, 6, 7, 8, 9, 10, 15, 20],
                'min_samples_leaf': [5, 10, 15, 20],
               }]

param_grid7 = [{'criterion': ['absolute_error'],
                'max_depth': [5, 6, 7, 8, 9, 10, 15, 20],
                'min_samples_leaf': [5, 10, 15, 20],
                'n_estimators': [5, 6, 7, 8, 9, 10, 20, 30, 40, 50]}]

### Setting up Grid Search for inner loops

In [19]:

gridcvs = {}

inner_cv = StratifiedKFold(n_splits=3, shuffle=True)

for pgrid, est, name in zip((param_grid1, param_grid2, param_grid3, param_grid4, param_grid5, param_grid6, param_grid7),
                            (reg1, reg2, reg3, reg4, reg5, reg6, reg7),
                            ('Ridge', 'Lasso', 'ElasticNet', 'KNN', 'SVR', 'DTree', 'RForest')):
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='neg_mean_squared_error',
                       n_jobs=-1,
                       cv=inner_cv,                       
                       verbose=0,
                       refit=True
                      )
    gridcvs[name] = gcv

In [20]:
gridcvs

{'Ridge': GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
              estimator=Ridge(), n_jobs=-1,
              param_grid=[{'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5,
                                     10, 15, 20]}],
              scoring='neg_mean_squared_error'),
 'Lasso': GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
              estimator=Lasso(), n_jobs=-1,
              param_grid=[{'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5,
                                     10, 15, 20]}],
              scoring='neg_mean_squared_error'),
 'ElasticNet': GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
              estimator=ElasticNet(), n_jobs=-1,
              param_grid=[{'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1, 5,
                                     10, 15, 20]}],
              scoring='neg_mean_squared_error'),
 'KNN': GridSearchCV(cv=StratifiedKFold(n_splits=3,

### Nested CV

In [22]:
import warnings
warnings.filterwarnings("ignore")

#pdb.set_trace()

bins = [10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = [1,2,3,4,5,6,7,8]
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels)

stratifyCategory = ['Sex', 'AgeGroup']

#from sklearn.model_selection import StratifiedShuffleSplit
#split = StratifiedShuffleSplit(test_size = 24, random_state=42, n_splits = 5)


feature_names = ['Age','Sex', 'EduYears', 'Smoker', 'High_BP', 'COMT', 'BDNF2', 'ApoE4',
                 'L_HH_Total', 'R_HH_Total', 'L_HB_Total', 'R_HB_Total', 'L_HT_Total', 'R_HT_Total',
                 'L_DG_Total', 'R_DG_Total',
                 'L_CA_Total', 'R_CA_Total',
                 'L_Sub_Total', 'R_Sub_Total',
                 'L_HH_CA', 'R_HH_CA', 'L_HB_CA', 'R_HB_CA', 'L_HT_CA', 'R_HT_CA', 
                 'L_HH_DG', 'R_HH_DG', 'L_HB_DG', 'R_HB_DG', 'L_HT_DG', 'R_HT_DG',
                 'L_HH_Sub', 'R_HH_Sub', 'L_HB_Sub', 'R_HB_Sub', 'L_HT_Sub', 'R_HT_Sub']


# Feature Scaling
scaler = MinMaxScaler()
df[feature_names] = scaler.fit_transform(df[feature_names])

for name, est in gridcvs.items():

        print(50 * '-', '\n')
        print('Algorithm:', name)

        outer_scores_mae = []
        outer_scores_mse = []
        outer_scores_rmse = []
        outer_scores_r2 = []
        
    
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True) 

        for train_index_outer, test_index_outer in outer_cv.split(df, df.grp):
            train_set = df.loc[train_index_outer,:]
            test_set = df.loc[test_index_outer,:]

            X = ['Age','Sex', 'EduYears', 
                     'L_HH_Total', 'R_HH_Total', 'L_HB_Total', 'R_HB_Total', 'L_HT_Total', 'R_HT_Total',
                     'L_DG_Total', 'R_DG_Total',
                     'L_CA_Total', 'R_CA_Total',
                     'L_Sub_Total', 'R_Sub_Total',
                     'L_HH_CA', 'R_HH_CA', 'L_HB_CA', 'R_HB_CA', 'L_HT_CA', 'R_HT_CA', 
                     'L_HH_DG', 'R_HH_DG', 'L_HB_DG', 'R_HB_DG', 'L_HT_DG', 'R_HT_DG',
                     'L_HH_Sub', 'R_HH_Sub', 'L_HB_Sub', 'R_HB_Sub', 'L_HT_Sub', 'R_HT_Sub']
            y = ['CVLT_Imm_Total']

            X_train = train_set[X]
            y_train = train_set[y]
            X_test = test_set[X]
            y_test = test_set[y]
                        
            #Apply grid search with CV=3 on outer train_set

            grid = gridcvs[name]
            grid.fit(X = X_train,
                     y = y_train) # run inner loop hyperparam tuning    
            print('\n        Best MSE (inner validation folds):', abs(grid.best_score_))
            print('        Best parameters:', grid.best_params_)

            #Inner Train set performance
            y_train_hat = grid.best_estimator_.predict(X_train)
            mse_train = mean_squared_error(y_train, y_train_hat)
            print('        MSE (on outer training fold)', mse_train)
            
            # Calculate evaluation metrics using best-tuned model on out val_set
            #MSE
            y_test_hat = grid.best_estimator_.predict(X_test)
            mse_test = mean_squared_error(y_test, y_test_hat)
            outer_scores_mse.append(mse_test)
            print('        MSE (on outer validation fold)', (outer_scores_mse[-1]))
                
            #RMSE
            rmse_test = np.sqrt(mse_test)
            outer_scores_rmse.append(rmse_test)
            print('        RMSE (on outer validation fold)', (outer_scores_rmse[-1]))
            
            # MAE
            mae_test = mean_absolute_error(y_test, y_test_hat)
            outer_scores_mae.append(mae_test)
            print('        MAE (on outer validation fold)', (outer_scores_mae[-1]))
            
#            # R2
#            r2_val = r2_score(y_val, y_val_hat)
#            outer_scores_r2.append(r2_val)
#            
#            print('        R2 (on outer validation fold)', (outer_scores_r2[-1]))
            
        print('\n    Outer Loop:')
        print('        MSE %.2f +/- %.2f'%
                  (np.mean(outer_scores_mse), np.std(outer_scores_mse)))
        print('        RMSE %.2f +/- %.2f'% 
                  (np.mean(outer_scores_rmse), np.std(outer_scores_rmse)))
        print('        MAE %.2f +/- %.2f'%
                  (np.mean(outer_scores_mae), np.std(outer_scores_mae)))
#        print('        R2 %.2f +/- %.2f'%
#                  (np.mean(outer_scores_r2), np.std(outer_scores_r2)))

-------------------------------------------------- 

Algorithm: Ridge

        Best MSE (inner validation folds): 70.61365476612283
        Best parameters: {'alpha': 5}
        MSE (on outer training fold) 58.22992067277527
        MSE (on outer validation fold) 39.900375701033006
        RMSE (on outer validation fold) 6.316674417843063
        MAE (on outer validation fold) 5.1938207266247565

        Best MSE (inner validation folds): 60.19877864904233
        Best parameters: {'alpha': 5}
        MSE (on outer training fold) 51.058574217591634
        MSE (on outer validation fold) 80.09329376073259
        RMSE (on outer validation fold) 8.949485670178627
        MAE (on outer validation fold) 7.342179230639579

        Best MSE (inner validation folds): 63.487354425684465
        Best parameters: {'alpha': 5}
        MSE (on outer training fold) 52.701142669702065
        MSE (on outer validation fold) 59.82309586322716
        RMSE (on outer validation fold) 7.734539150022266
 


        Best MSE (inner validation folds): 65.07720047952448
        Best parameters: {'C': 30, 'degree': 1, 'gamma': 'auto', 'kernel': 'rbf'}
        MSE (on outer training fold) 50.44572061329277
        MSE (on outer validation fold) 83.9509119430602
        RMSE (on outer validation fold) 9.162473025502461
        MAE (on outer validation fold) 7.423249179314899

        Best MSE (inner validation folds): 58.00746354145432
        Best parameters: {'C': 30, 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
        MSE (on outer training fold) 3.5220862864860845
        MSE (on outer validation fold) 78.60525423038102
        RMSE (on outer validation fold) 8.86596042346124
        MAE (on outer validation fold) 7.152945284348334

    Outer Loop:
        MSE 73.37 +/- 13.76
        RMSE 8.52 +/- 0.87
        MAE 6.95 +/- 0.54
-------------------------------------------------- 

Algorithm: DTree

        Best MSE (inner validation folds): 93.7608543417367
        Best parameters: {'cr